In [ ]:
!apt install tesseract-ocr -y
!pip install pytesseract opencv-python pillow crewai langchain huggingface_hub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.5 MB/s eta 0:00:00
   ━━━

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

Document Verify

In [ ]:
from google.colab import files
uploaded = files.upload()

# Save as a file and get the path
image_path = list(uploaded.keys())[0]


Saving sample2.jpeg to sample2.jpeg


In [ ]:
pip install -U langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
import cv2
import os
from crewai import Agent
from langchain_community.llms import HuggingFaceEndpoint
import pytesseract

class DocumentCheckingAgent:
    def __init__(self, tesseract_path=None):
        if tesseract_path:
            pytesseract.pytesseract.tesseract_cmd = tesseract_path

        self.agent = Agent(
            role="Document Checker",
            goal="Verify submitted student documents for admission.",
            backstory="You analyze scanned student documents...",
            llm=HuggingFaceEndpoint(
                endpoint_url="https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-alpha",
                #huggingfacehub_api_token="",  # Optional unless using a private model
                temperature=0.2
            ),
            verbose=True
        )

    def extract_text_from_image(self, image_path):
        if not os.path.exists(image_path):
            return "Error: File not found."

        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        text = pytesseract.image_to_string(gray)
        return text.strip()

    def validate_document(self, image_path, expected_keywords):
        extracted_text = self.extract_text_from_image(image_path)
        if extracted_text == "Error: File not found.":
            return {"status": "error", "message": "Document not found."}

        missing_keywords = [word for word in expected_keywords if word.lower() not in extracted_text.lower()]
        if missing_keywords:
            return {"status": "failed", "message": f"Missing data: {', '.join(missing_keywords)}"}
        return {"status": "verified", "message": "Document is valid."}


In [ ]:
doc_checker = DocumentCheckingAgent("/usr/bin/tesseract")  # Set path explicitly
result = doc_checker.validate_document(
    image_path,
    ["Name", "Registration No", "Overall Grade", "Result", "Roll No"]
)
print(result)


Provider List: https://docs.litellm.ai/docs/providers

{'status': 'verified', 'message': 'Document is valid.'}


**Shortlisting Agent** -



*   Overall Grade must be B or higher (B, B+, A, A+)
*   No key document field missing
*   Result is PASS

In [ ]:
pip install -U langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain cohere streamlit python-dotenv chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
from langchain.llms import Cohere  # Updated import for Cohere

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Initialize Cohere with the API key loaded from the environment
llm = Cohere(cohere_api_key=os.getenv("COHERE_API_KEY"))

In [ ]:
from langchain.agents import initialize_agent, AgentType

In [ ]:
import json

# Load the FAQ chain
def load_faq_chain():
    with open("faq_data.json", "r") as f:
        faqs = json.load(f)

    docs = [Document(page_content=f"{faq['question']} - {faq['answer']}") for faq in faqs]
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory="./chroma_db"
    )

    qa_chain = load_qa_chain(llm=llm, chain_type="stuff")

    return RetrievalQA(
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        combine_documents_chain=qa_chain
    )

faq_chain = load_faq_chain()

# Define FAQ tool
faq_tool = Tool(
    name="FAQ Retriever",
    func=lambda q: faq_chain.run(q),
    description="Use this to answer FAQs related to student admission."
)

<ipython-input-22-f4fe48940b5f>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-22-f4fe48940b5f>:16: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm=llm, chain_type="stuff")
<ipython-input-22-f4fe48940b5f>:18: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  return RetrievalQA(


In [ ]:
extracted_text = doc_checker.extract_text_from_image(image_path)

In [ ]:
import re

def parse_extracted_text(extracted_text):
    # Extract result (PASS/FAIL)
    result_match = re.search(r"Grade\s+([A-Z\+]+)", extracted_text)  # Capture result (PASS/FAIL)
    result = result_match.group(1) if result_match else None

    # Extract overall grade (B, A, B+ etc.)
    grade_match = re.search(r"PASS\s+\d+\s+([A-Z\+]+)", extracted_text)  # Capture grade (B, A, B+)
    overall_grade = grade_match.group(1) if grade_match else None

    return {
        "result": result,
        "overall_grade": overall_grade
    }

# Parse the extracted text
parsed_data = parse_extracted_text(extracted_text)

# Print the parsed data
print(parsed_data)

{'result': 'PASS', 'overall_grade': 'B'}


In [ ]:
import re
import json

class ShortlistingAgent:
    def __init__(self, accepted_grades=["B","B+", "A", "A+"]):
        self.accepted_grades = accepted_grades

    def shortlist(self, input_str):
        try:
            # Parse the input JSON string into Python data structure (dict)
            data = json.loads(input_str)

            # Extract verification result and extracted text
            verification_result = data.get("verification_result") or data.get("verified_document")
            extracted_text = data.get("extracted_text")

            # If not available, try reconstructing from other keys
            if isinstance(extracted_text, dict):
                overall_grade = extracted_text.get('overall_grade')
                result = extracted_text.get('result')
                if overall_grade:
                    extracted_text = f"Result: {result}\nOverall Grade: {overall_grade}"

            if not verification_result:
                return "Rejected: No verification info provided."

            if verification_result.get("status") != "verified":
                return f"Rejected: {verification_result.get('message', 'Document verification failed.')}"

            # Extract grade information from the extracted_text
            lines = [line.strip() for line in extracted_text.split('\n') if line.strip()]

            # Print lines to ensure we're extracting the correct information
            print(f"Lines after splitting extracted text: {lines}")

            grade = None

            # Try to extract the overall grade
            for line in lines:
                if "Overall Grade" in line:
                    # Use regex to cleanly extract the grade after "Overall Grade"
                    match = re.search(r"Overall Grade\s*[:\-]?\s*(\w+)", line)
                    if match:
                        grade = match.group(1)  # Extracted grade
                    break

            # If grade is not found, return rejection
            if not grade:
                return f"Rejected: Grade not found in the extracted text."

            # Print the extracted grade for debugging
            print(f"Extracted Grade: {grade}")

            # Validate the grade
            if grade in self.accepted_grades:
                return "Shortlisted"
            else:
                return f"Rejected: Grade '{grade}' not accepted."

        except Exception as e:
            return f"Error processing the input: {str(e)}"

# Wrap shortlisting agent in a Tool
shortlist_agent = ShortlistingAgent()

shortlisting_tool = Tool(
    name="Shortlisting Agent",
    func=lambda query: shortlist_agent.shortlist(query),
    description="Use this to decide if a student should be shortlisted based on verified document and grade info."
)

# Create the agent
tools = [faq_tool, shortlisting_tool]

agent_executor = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True  # Add this parameter
)

# Define the Shortlisting Tool (assuming you have `Tool` and `agent_executor` properly set up)
shortlisting_tool = Tool(
    name="Shortlisting Agent",
    func=lambda query: shortlist_agent.shortlist(query),
    description="Use this to decide if a student should be shortlisted based on verified document and grade info."
)

# Create query data
query_data = {
    "verification_result": result,
    "extracted_text": parsed_data
}

# Assuming agent_executor is properly initialized
response = agent_executor.run(f"Shortlist this student: {json.dumps(query_data)}")
print(response)

<ipython-input-23-77942cb5b82c>:75: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(
<ipython-input-23-77942cb5b82c>:97: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor.run(f"Shortlist this student: {json.dumps(query_data)}")




> Entering new AgentExecutor chain...
 This is the expected format for a student being reviewed for shortlisting, meaning I should shortlist them.
Action: Shortlisting Agent
Action Input: {"verification_result": {"status": "verified", "message": "Document is valid."}, "extracted_text": {"result": "PASS", "overall_grade": "B"}}Lines after splitting extracted text: ['Result: PASS', 'Overall Grade: B']
Extracted Grade: B

Observation: Shortlisted
Thought: This student has the required score and their document has been verified, so they meet the criteria to be shortlisted.
Action: Shortlisting Agent
Action Input: {"verification_result": {"status": "verified", "message": "Document is valid."}, "extracted_text": {"result": "PASS", "overall_grade": "B"}}Lines after splitting extracted text: ['Result: PASS', 'Overall Grade: B']
Extracted Grade: B

Observation: Shortlisted
Thought: This student is also eligible for shortlisted status therefore meeting the set criteria for this process. 
Actio

In [ ]:
print(cleaned_response)

Shortlisted


In [ ]:
cleaned_response = response.replace('\n', ' ').strip()

if cleaned_response.rstrip('.') == "Shortlisted":
    print(f"Final Answer: {cleaned_response.strip()}")
else:
    print("No final answer found.")

Final Answer: Shortlisted


In [ ]:
print(extracted_text)

Roll No
Registration No
Name

Father's Name
Mother's Name
School/District
Stream

set

Category

Subject
GENERAL ENGLISH
GENERAL PUNJABI
PHYSICS
CHEMISTRY
BIOLOGY

COMPUTER SCIENCE*

Candidate's Particular

 

-
(AMRITSAR)
Science
Regular (General Fresh Candidates)
Marks Det
Th «CCE Pr_~—s Total ~—sResult
042 008 = Xxx 050 P

052 009 XXX 061.
023 007 018 048.

e

e
024 006 016 +046 p
e

029 009 016 = 054

036 008 033 077

ENVIRONMENTAL EDUCATION™ 034 005 = XXX 039

RESULT

Overall Grade

PASS 259
e

Grade

ona0p


In [ ]:
print(type(result))

<class 'dict'>


In [ ]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.agents import Tool

In [ ]:
print(extracted_text)

saa.
u/s a\Y
% CS
2

 

Roll No
Registration No
Name

Father's Name
Mother's Name
School/District
Stream

Set

Category

Subject
GENERAL ENGLISH
GENERAL PUNJABI

Punjab School Education Board
Senior Secondary (10+2) Examination Provisional Result March 2024

Candidate's Particular

BS/19/80:

S.F.H.J.L.GOVT. SENIOR SECONDARY SCHOOL, MIRTHAL (PATHANKOT)
Commerce
$-11 (0172-5227315)
Regular (General Fresh Candidates)
Marks Details

Th INA Pr Total Result Grade
056 017 XXX 073 Pp B+
060 016 XXX 076 Pp B+

BUSINESS STUDIES-II 038 020 XXX 058 Pp C+
ACCOUNTANCY-II 047 005 014 066 P B
ECONOMICS 056 015 XXX O71 . B+
*ENVIRONMENTAL EDUCATION 035 010 XXX 045 Pp A
*COMPUTER SCIENCE 041 005 040 086 , A

RESULT
Overall Grade

PASS 344
B
Declared on 30 April 2024


**Student Loan Agent**

*   verified Documents
*   shortlisted
*   requested loan amount
*   fixed university budget
*   Income Statement



In [ ]:
class StudentLoanAgent:
    def __init__(self, university_budget):
        self.total_budget = university_budget
        self.remaining_budget = university_budget
        self.approved_loans = []

    def process_loan_request(self, student_name, verification_result, shortlisting_result, requested_amount):
        # Check verification and shortlisting
        if verification_result["status"] != "verified":
            return {"status": "rejected", "reason": "Document verification failed."}

        if shortlisting_result["status"] != "shortlisted":
            return {"status": "rejected", "reason": "Student not shortlisted."}

        # Check budget
        if requested_amount > self.remaining_budget:
            return {
                "status": "rejected",
                "reason": f"Insufficient university budget. Only ₹{self.remaining_budget} available."
            }

        # Approve the loan
        self.remaining_budget -= requested_amount
        self.approved_loans.append({
            "student": student_name,
            "amount": requested_amount
        })

        return {
            "status": "approved",
            "student": student_name,
            "amount": requested_amount,
            "message": f"Loan approved! Remaining university budget: ₹{self.remaining_budget}"
        }

    def summary(self):
        return {
            "total_budget": self.total_budget,
            "remaining_budget": self.remaining_budget,
            "approved_loans": self.approved_loans
        }

In [ ]:
loan_agent = StudentLoanAgent(university_budget=100000) #1 lakh

In [ ]:
loan_result = loan_agent.process_loan_request(
    student_name="ABC",
    verification_result=result,  # from DocumentCheckingAgent
    shortlisting_result=decision,  # from ShortlistingAgent
    requested_amount=25000
)

print(loan_result)

{'status': 'approved', 'student': 'ABC', 'amount': 25000, 'message': 'Loan approved! Remaining university budget: ₹75000'}


In [ ]:
loan_result = loan_agent.process_loan_request(
    student_name="ABC",
    verification_result=result,  # from DocumentCheckingAgent
    shortlisting_result=decision,  # from ShortlistingAgent
    requested_amount=250000
)

print(loan_result)

{'status': 'rejected', 'reason': 'Insufficient university budget. Only ₹75000 available.'}


**AGENT**

In [ ]:
def load_faq_chain():
    with open("loan_data.json", "r") as f:
        faqs = json.load(f)

    docs = [Document(page_content=f"{faq['question']} - {faq['answer']}") for faq in faqs]
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory="./loan_faq_db"
    )

    qa_chain = load_qa_chain(llm=llm, chain_type="stuff")

    return RetrievalQA(
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        combine_documents_chain=qa_chain
    )

faq_chain = load_faq_chain()

faq_tool = Tool(
    name="Loan FAQ Retriever",
    func=lambda q: faq_chain.run(q),
    description="Use this to answer FAQs related to student loan approval."
)

In [ ]:
class LoanDecisionAgent:
    def __init__(self, budget, income_threshold):
        self.budget = budget
        self.income_threshold = income_threshold

    def approve_loan(self, input_str):
        try:
            data = json.loads(input_str)

            # Extract inputs
            shortlisted = data.get("shortlisted", "").strip().lower()
            annual_income = float(data.get("annual_income", 0))
            requested_loan = float(data.get("requested_loan", 0))

            temp_budget = self.budget  # Avoid mutation unless approved in final output

            if shortlisted != "shortlisted":
                return "Loan Rejected: Student not shortlisted."

            if annual_income > self.income_threshold:
                return f"Loan Rejected: Annual income {annual_income} exceeds threshold {self.income_threshold}."

            if requested_loan > temp_budget:
                return f"Loan Rejected: Requested loan ({requested_loan}) exceeds remaining budget ({temp_budget})."

            # Simulate approval, don't mutate real budget during intermediate calls
            return f"Loan Approved for ₹{requested_loan}. Remaining Budget: ₹{temp_budget - requested_loan}"

        except Exception as e:
            return f"Error processing loan application: {str(e)}"

    def finalize_approval(self, input_str):
        """Deduct from real budget after final decision"""
        data = json.loads(input_str)
        requested_loan = float(data.get("requested_loan", 0))
        self.budget -= requested_loan
        return f"[FINAL] Loan Approved. Remaining Budget: ₹{self.budget}"

In [ ]:
loan_agent = LoanDecisionAgent(budget=500000, income_threshold=300000)

In [ ]:
loan_tool = Tool(
    name="Student Loan Agent",
    func=lambda query: loan_agent.approve_loan(query),
    description="Use this to decide if a student is eligible for a loan based on shortlisting, income, and university budget."
)

# Register tools with agent
tools = [faq_tool, loan_tool]

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
print(cleaned_response)

Shortlisted


In [ ]:
annual_income = int(input("Enter your annual income (in INR): "))
requested_loan_amount = int(input("Enter requested loan amount (in INR): "))

query_loan={
    "shortlisted": cleaned_response,   # You can pass in the extr10acted cleaned_response
    "annual_income": annual_income,        # From user input
    "requested_loan": requested_loan_amount
}

response = agent_executor.run(f"Evaluate loan eligibility: {json.dumps(query_loan)}")
print(response)

Enter your annual income (in INR): 60000
Enter requested loan amount (in INR): 600000


> Entering new AgentExecutor chain...
 This is a case where we have all the information needed to make a decision and take action. Let's evaluate the eligibility of the student loan requested by the given criteria to determine the next steps for the client. 
Action: Student Loan Agent
Action Input: {"shortlisted": "Shortlisted", "annual_income": 60000, "requested_loan": 600000}
Observation: Loan Rejected: Requested loan (600000.0) exceeds remaining budget (500000).
Thought: The student's requested loan exceeds the remaining budget for their university. This means they are not eligible for the amount they want. I should inform them of the maximum eligible amount they could apply for. 
Action: Loan FAQ Retriever
Action Input: maximum loan amount eligibility
Observation:  The maximum loan amount that a student can be eligible for depends on the university's budget and the student's financial need, as d